In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df=pd.read_csv("../input/housesalesprediction/kc_house_data.csv")

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
# no missing data

In [ ]:
df.info()

In [ ]:
df.describe().transpose()

# Exploratory Data Analysis

In [ ]:
plt.figure(figsize=(14,8))
sns.distplot(df['price'],bins=40)

In [ ]:
# it shows most of the houses range below 20,00,000..so its better to exclude the outliers while creating the model

In [ ]:
sns.countplot(df['bedrooms'])

In [ ]:
# most of the houses have 2-5 bedrooms

In [ ]:
df.corr()['price'].sort_values()

In [ ]:
plt.figure(figsize=(14,8))
sns.scatterplot(x='price',y='sqft_living',data=df)

In [ ]:
plt.figure(figsize=(14,6))
sns.scatterplot(x='price',y='lat',data=df)

In [ ]:
plt.figure(figsize=(14,6))
sns.scatterplot(x='price',y='long',data=df)

In [ ]:
# at a certain combination of latitude and longitude we have an expensive housing area

In [ ]:
plt.figure(figsize=(14,8))
sns.scatterplot(x='long',y='lat',data=df,hue='price')

In [ ]:
# it rersembles the map of king county

In [ ]:
df.sort_values('price',ascending=False)

In [ ]:
# it is better to drop the outliers

In [ ]:
len(df)*0.01

In [ ]:
# so 20 is less than 1% thus we can drop them and it wont make much of a difference

In [ ]:
bottom_99_perc=df.sort_values('price',ascending=False).iloc[216:]

In [ ]:
plt.figure(figsize=(14,8))
sns.scatterplot(x='long',y='lat',edgecolor=None,alpha=0.2,palette='viridis',data=bottom_99_perc,hue='price')

In [ ]:
# properties near water are more expensive

In [ ]:
plt.figure(figsize=(12,10))
sns.boxplot(x='waterfront',y='price',data=bottom_99_perc)

# feature engineering

In [ ]:
df.head()

In [ ]:
df=df.drop('id',axis=1)

In [ ]:
df['date']=pd.to_datetime(df['date'])

In [ ]:
df['date']

In [ ]:
df['year']=df['date'].apply(lambda date:date.year)

In [ ]:
df['month']=df['date'].apply(lambda date:date.month)

In [ ]:
plt.figure(figsize=(14,10))
sns.boxplot(x='month',y='price',data=df)

In [ ]:
# not a significant difference as far as month is considered

In [ ]:
df.groupby('year').mean()['price'].plot()

In [ ]:
# it makes sense considering the inflation

In [ ]:
df=df.drop('date',axis=1)

In [ ]:
df.columns

In [ ]:
df=df.drop('zipcode',axis=1)

# model training

In [ ]:
X=df.drop('price',axis=1).values
y=df['price'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=101)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()

In [ ]:
X_train=scaler.fit_transform(X_train)

In [ ]:
# we dont fit to our test set as we dont want prior info of test set and prevent data leakage
X_test=scaler.transform(X_test)

# creating the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
X_train.shape

In [ ]:
model=Sequential()

model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))

model.add(Dense(1))


model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
         batch_size=128,
         epochs=400)

In [ ]:
losses=pd.DataFrame(model.history.history)

In [ ]:
losses.columns

In [ ]:
plt.figure(figsize=(12,6))
losses.plot()

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,explained_variance_score

In [ ]:
predictions=model.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,predictions))

In [ ]:
mean_absolute_error(y_test,predictions)

In [ ]:
explained_variance_score(y_test,predictions)

In [ ]:
# its not that good and not that bad either

# testing on new value

In [ ]:
single_house=df.drop('price',axis=1).iloc[0]

In [ ]:
single_house=scaler.transform(single_house.values.reshape([-1,19]))

In [ ]:
model.predict(single_house)